In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
import submit_code_tool
import query_tool
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.memory import ConversationBufferMemory


In [4]:
tools = [
   query_tool.reference_code_search,
   submit_code_tool.create_submission_tool("my_session")
  ]

message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history, return_messages=True, human_prefix="human", ai_prefix="ai", input_key="input", output_key="output"
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are an AWS solutions architect. Your job is to help customers build and deploy secure applications on AWS. Follow architecture best practices and find out approriate requirements before suggesting a solution. Once a tool call fails, stop trying. Once you have enough information, submit a proposal for a modified terraform file using the submit code tool. Try to keep the terraform file as similar to the original as possible."),
  ("human", "My current terraform file:\n{tf_file}"),
  MessagesPlaceholder("chat_history", optional=True),
  ("human", "{input}"),
  "{agent_scratchpad}"
])


llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=.5)
# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt) # type: ignore # wrong type for query_tool import

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, max_iterations=8, early_stopping_method="generate", return_intermediate_steps=True) # type: ignore # wrong type for query_tool import

In [8]:
import redis
r = redis.Redis(host="localhost", port=6379, decode_responses=True)
messages = r.lrange("message_store:my-session", 0, -1)
messages

['{"type": "ai", "data": {"content": "I have created a Terraform configuration for a managed PostgreSQL database instance on AWS RDS with daily backups and without high availability requirements. Here is the modified Terraform file:\\n\\n```terraform\\n# Configure the AWS provider\\nprovider \\"aws\\" {\\n  region = \\"eu-west-1\\"\\n}\\n\\n# Create a DB instance\\nresource \\"aws_db_instance\\" \\"example\\" {\\n  engine                   = \\"postgres\\"\\n  allocated_storage        = 20\\n  instance_class           = \\"db.t2.micro\\"\\n  name                     = \\"example_database_stage\\"\\n  username                 = \\"admin\\"\\n  password                 = \\"${var.db_password}\\"\\n  skip_final_snapshot      = false\\n  backup_retention_period  = 7\\n}\\n```\\n\\nThis configuration sets up a PostgreSQL database with a `db.t2.micro` instance class, 20 GB of allocated storage, and a backup retention period of 7 days, which means it will have daily backups for one week. The 

In [5]:
agent_executor.invoke({
  "input": "I need a manged postgres database without specific requirements. No high availability, but at least daily backups.",
  "tf_file": "```terraform resource \"aws_instance\" \"example\" { ami = \"${data.aws_ami.ubuntu.id}\" instance_type = \"t3.micro\" }```",
})



> Entering new AgentExecutor chain...

Invoking: `reference_code_search` with `{'query': 'AWS RDS Postgres Terraform'}`


# Configure the AWS provider
provider "aws" {
  region = "eu-west-1"
}

# Create a DB instance
resource "aws_db_instance" "example" {
  engine              = "mysql"
  allocated_storage   = 10
  instance_class      = "db.t2.micro"
  name                = "example_database_stage"
  username            = "admin"
  password            = "${var.db_password}"
  skip_final_snapshot = true
}

Invoking: `submit_modified_tf` with `{'new_code': '# Configure the AWS provider\nprovider "aws" {\n  region = "eu-west-1"\n}\n\n# Create a DB instance\nresource "aws_db_instance" "example" {\n  engine              = "postgres"\n  allocated_storage   = 20\n  instance_class      = "db.t2.micro"\n  name                = "example_database_stage"\n  username            = "admin"\n  password            = "${var.db_password}"\n  skip_final_snapshot = false\n  backup_retention_period = 7\n}

{'input': 'I need a manged postgres database without specific requirements. No high availability, but at least daily backups.',
 'tf_file': '```terraform resource "aws_instance" "example" { ami = "${data.aws_ami.ubuntu.id}" instance_type = "t3.micro" }```',
 'chat_history': [],
 'output': 'I have created a Terraform configuration for a managed PostgreSQL database instance on AWS RDS with daily backups and without high availability requirements. Here is the modified Terraform file:\n\n```terraform\n# Configure the AWS provider\nprovider "aws" {\n  region = "eu-west-1"\n}\n\n# Create a DB instance\nresource "aws_db_instance" "example" {\n  engine                   = "postgres"\n  allocated_storage        = 20\n  instance_class           = "db.t2.micro"\n  name                     = "example_database_stage"\n  username                 = "admin"\n  password                 = "${var.db_password}"\n  skip_final_snapshot      = false\n  backup_retention_period  = 7\n}\n```\n\nThis configurati